In [1]:
##RandomForest
#@author:angela 

#Import functions needed
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score 
from sklearn.utils.class_weight import compute_class_weight
from matplotlib.lines import Line2D



In [2]:
#Current directory
os.getcwd()
#Change working directory
os.chdir('C:\\Users\\Administrador\\Desktop\\python')
os.listdir()

['.ipynb_checkpoints',
 'data.csv',
 'graficos',
 'Model_6800 pairs of DNA.ipynb',
 'model_DecisionTreeClassifier-checkpoint.ipynb',
 'model_hypercholesterolemia-checkpoint.ipynb',
 'model_LDA.ipynb']

In [3]:
#Loading data: 
pd.read_csv("data.csv", sep=";")
data = pd.read_csv("data.csv",sep=";")
data.head(10) #Shows 10 first rows from de dataset

,n,f(vij/c0)obj,Subset,f(c0)ref,D3(Set1),V2(Set1),D3(Set2),V1(Set2),D3(Set3),D1(Set4)
0,1,0,t,0.325301,2.515821,0.24,3.37,0.1,3.03,58.64
1,2,0,t,0.325301,2.475821,0.24,3.33,0.1,2.99,120.59
2,3,0,t,0.325301,2.475821,-1.01,3.33,0.1,2.99,115.82
3,4,0,v,0.325301,1.735821,0.24,2.59,0.1,2.25,7.90
4,5,0,t,0.325301,0.955821,0.24,1.81,0.1,1.47,2.53
5,6,0,t,0.325301,0.805821,0.24,1.66,0.1,1.32,-20.15
6,7,0,t,0.325301,0.175821,0.24,1.03,0.1,0.69,-25.52
7,8,0,v,0.325301,-0.194179,-1.01,0.66,0.1,0.32,24.49
8,9,0,t,0.325301,-0.194179,0.24,0.66,0.1,0.32,24.49
9,10,0,t,0.325301,-0.604179,0.24,0.25,0.1,-0.09,-53.58


In [4]:
#Data Splitting:
Y= data.iloc[:,1]
X = data.iloc[:, 3:]
print(Y) 
print(X)

0         0
1         0
2         0
3         0
4         0
         ..
369253    0
369254    0
369255    0
369256    0
369257    0
Name: f(vij/c0)obj, Length: 369258, dtype: int64
        f(c0)ref  D3(Set1)  V2(Set1)  D3(Set2)  V1(Set2)  D3(Set3)  D1(Set4)
0       0.325301  2.515821      0.24      3.37       0.1      3.03     58.64
1       0.325301  2.475821      0.24      3.33       0.1      2.99    120.59
2       0.325301  2.475821     -1.01      3.33       0.1      2.99    115.82
3       0.325301  1.735821      0.24      2.59       0.1      2.25      7.90
4       0.325301  0.955821      0.24      1.81       0.1      1.47      2.53
...          ...       ...       ...       ...       ...       ...       ...
369253  0.000000 -1.889286     -0.07     -1.50       0.1     -1.42    -17.50
369254  0.000000 -1.889286     -0.07     -1.50       0.1     -1.42    -17.50
369255  0.000000 -2.209286     -0.07     -1.82       0.1     -1.74    -61.52
369256  0.000000 -2.209286     -0.07     -1.82   

In [5]:
#Splitting Data into Training and Prediction Sets:
train_data = data[data["Subset"] == "t"]
pred_data = data[data["Subset"] == "v"]

#Model Preparation for Random Forest:
X_train_forest= train_data.iloc[:, 3:]
X_test_forest = pred_data.iloc[:, 3:]
y_train_forest= train_data.iloc[:, 1]
y_test_forest= pred_data.iloc[:, 1]

print(X_test_forest.shape)
print(X_train_forest.shape)
print(y_test_forest.shape)
print(y_train_forest.shape)

(92314, 7)
(276944, 7)
(92314,)
(276944,)


In [6]:
#RANDOM FOREST MODEL
#X_forest
mlr_X_train_forest= X_train_forest[['f(c0)ref', 'D3(Set1)', 'V2(Set1)','D3(Set2)','V1(Set2)','D3(Set3)','D1(Set4)']]
mlr_X_test_forest = X_test_forest[['f(c0)ref', 'D3(Set1)', 'V2(Set1)','D3(Set2)','V1(Set2)','D3(Set3)','D1(Set4)']]

#Define prior probabilities and adjust as needed
class_weight = {0:0.5,1:0.5}
# Define the random forest model
seed = 44
rf_model = RandomForestClassifier(random_state=seed, class_weight=class_weight)

In [7]:
#Define the hyperparameter search space
param_grid = {
    'n_estimators': [50],
    'max_depth': [10],
    'min_samples_split': [6],
    'min_samples_leaf': [2]
}

# Perform hyperparameter search by cross-validation
grid_search_forest = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# fit model
grid_search_forest.fit(mlr_X_train_forest, y_train_forest)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight={0: 0.5, 1: 0.5},
                                              random_state=44),
             n_jobs=-1,
             param_grid={'max_depth': [10], 'min_samples_leaf': [2],
                         'min_samples_split': [6], 'n_estimators': [50]},
             scoring='accuracy')

In [8]:
#make predictions on the train set
y_pred_train_forest = grid_search_forest.predict(mlr_X_train_forest)

In [9]:
#calculate confusion matrix
cm_train_forest = confusion_matrix(y_train_forest, y_pred_train_forest)
print(cm_train_forest)

#calculate performance metrics
tn_train_forest, fp_train_forest, fn_train_forest, tp_train_forest = cm_train_forest.ravel()

specificity_train_forest = tn_train_forest / (tn_train_forest + fp_train_forest)
sensitivity_train_forest = tp_train_forest / (tp_train_forest + fn_train_forest)
accuracy_train_forest = (tp_train_forest + tn_train_forest) / (tp_train_forest + tn_train_forest + fp_train_forest + fn_train_forest)
false_positive_train_forest = fp_train_forest / (fp_train_forest + tn_train_forest)
print('Specificity:',specificity_train_forest,'\n', 'Sensitivity:', sensitivity_train_forest,'\n','Accuracy:', accuracy_train_forest)
mcc_train_forest = matthews_corrcoef(y_train_forest, y_pred_train_forest)

[[ 57209  35649]
 [ 29625 154461]]
Specificity: 0.6160912360808977 
 Sensitivity: 0.8390697826016101 
 Accuracy: 0.7643061413137674


In [10]:
#make predictions on the test set
y_pred_test_forest = grid_search_forest.predict(mlr_X_test_forest)

In [11]:
#calculate confusion matrix
cm_test_forest = confusion_matrix(y_test_forest, y_pred_test_forest)
print(cm_train_forest)

#calculate performance metrics
tn_test_forest, fp_test_forest, fn_test_forest, tp_test_forest = cm_test_forest.ravel()

specificity_test_forest = tn_test_forest / (tn_test_forest + fp_test_forest)
sensitivity_test_forest = tp_test_forest / (tp_test_forest + fn_test_forest)
accuracy_test_forest = (tp_test_forest + tn_test_forest) / (tp_test_forest + tn_test_forest + fp_test_forest + fn_test_forest)
false_positive_test_forest = fp_test_forest / (fp_test_forest + tn_test_forest)
print('Specificity:',specificity_test_forest,'\n', 'Sensitivity:', sensitivity_test_forest,'\n','Accuracy:', accuracy_test_forest)
mcc_test_forest = matthews_corrcoef(y_test_forest, y_pred_test_forest)

[[ 57209  35649]
 [ 29625 154461]]
Specificity: 0.6113499014316647 
 Sensitivity: 0.8380994280686318 
 Accuracy: 0.7620945902030027
